In [1]:
import sklearn
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\Jason
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Jason
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Jason
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Jason
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [3]:
print(len(train))
train.head(10)
test.head()

7613


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [4]:
train = train.drop(columns=['id', 'keyword', 'location'])
IDs = test['id']
test = test.drop(columns=['id', 'keyword', 'location'])
train = train.drop_duplicates()
train['text'] = train['text'].str.lower()
test.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
train['tokens'] = train.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
test['tokens'] = test.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

In [6]:
train.isna().sum()

text      0
target    0
tokens    0
dtype: int64

In [7]:
test.isna().sum()

text      0
tokens    0
dtype: int64

In [8]:
# Make a list of english stopwords
stopwords = nltk.corpus.stopwords.words("english")

# Extend the list with your own custom stopwords, will use when vectorizing
my_stopwords = ['https', 'http', '...', 'The', "n't"]
stopwords.extend(my_stopwords)
#remove infrequent words
train['strings'] = train['tokens'].apply(lambda x: ' '.join([item for item in x if (len(item)>2) and (item not in stopwords)]))
test['strings'] = test['tokens'].apply(lambda x: ' '.join([item for item in x if (len(item)>2) and (item not in stopwords)]))
train.head()

,text,target,tokens,strings
0,our deeds are the reason of this #earthquake m...,1,"[our, deeds, are, the, reason, of, this, #, ea...",deeds reason earthquake may allah forgive
1,forest fire near la ronge sask. canada,1,"[forest, fire, near, la, ronge, sask, ., canada]",forest fire near ronge sask canada
2,all residents asked to 'shelter in place' are ...,1,"[all, residents, asked, to, 'shelter, in, plac...",residents asked 'shelter place notified office...
3,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #, wildfires, evacua...","13,000 people receive wildfires evacuation ord..."
4,just got sent this photo from ruby #alaska as ...,1,"[just, got, sent, this, photo, from, ruby, #, ...",got sent photo ruby alaska smoke wildfires pou...


In [9]:
# get all the tokenized words from df
all_words = ' '.join([word for word in train['strings']])
tokenized_words = nltk.tokenize.word_tokenize(all_words)

from nltk.probability import FreqDist

# see frequency of words
fdist = FreqDist(tokenized_words)
fdist

FreqDist({'like': 345, 'amp': 342, 'fire': 244, 'get': 228, 'new': 222, 'via': 215, 'news': 196, 'people': 194, 'one': 194, 'video': 162, ...})

In [10]:
# keep words with more than 5 appearance
train['text_string_fdist'] = train['tokens'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 5 ]))
test['text_string_fdist'] = test['tokens'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 5 ]))
train.head(10)

,text,target,tokens,strings,text_string_fdist
0,our deeds are the reason of this #earthquake m...,1,"[our, deeds, are, the, reason, of, this, #, ea...",deeds reason earthquake may allah forgive,reason earthquake may allah
1,forest fire near la ronge sask. canada,1,"[forest, fire, near, la, ronge, sask, ., canada]",forest fire near ronge sask canada,forest fire near . canada
2,all residents asked to 'shelter in place' are ...,1,"[all, residents, asked, to, 'shelter, in, plac...",residents asked 'shelter place notified office...,residents asked place ' officers . evacuation ...
3,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #, wildfires, evacua...","13,000 people receive wildfires evacuation ord...",people wildfires evacuation orders california
4,just got sent this photo from ruby #alaska as ...,1,"[just, got, sent, this, photo, from, ruby, #, ...",got sent photo ruby alaska smoke wildfires pou...,got sent photo alaska smoke wildfires school
5,#rockyfire update => california hwy. 20 closed...,1,"[#, rockyfire, update, =, >, california, hwy, ...",rockyfire update california hwy closed directi...,update california hwy . closed due lake county...
6,#flood #disaster heavy rain causes flash flood...,1,"[#, flood, #, disaster, heavy, rain, causes, f...",flood disaster heavy rain causes flash floodin...,flood disaster heavy rain causes flash floodin...
7,i'm on top of the hill and i can see a fire in...,1,"[i, 'm, on, top, of, the, hill, and, i, can, s...",top hill see fire woods,top hill see fire
8,there's an emergency evacuation happening now ...,1,"[there, 's, an, emergency, evacuation, happeni...",emergency evacuation happening building across...,emergency evacuation happening building across...
9,i'm afraid that the tornado is coming to our a...,1,"[i, 'm, afraid, that, the, tornado, is, coming...",afraid tornado coming area,afraid tornado coming area


In [11]:
wordnet_lem = WordNetLemmatizer()

train['text_string_lem'] = train['text_string_fdist'].apply(wordnet_lem.lemmatize)
test['text_string_lem'] = test['text_string_fdist'].apply(wordnet_lem.lemmatize)

In [12]:
X = train['text_string_lem']
y = train['target']

idf = TfidfVectorizer()
clf = Pipeline([('TF-IDF',idf), ('SVM', svm.SVC())])

clf.fit(X,y)
y_pred = clf.predict(test['text_string_lem'])


In [13]:
submit = pd.DataFrame()
submit['id'] = IDs
submit['target'] = y_pred

In [14]:
submit

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,0
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,0


In [15]:
submit.to_csv('submission.csv', index=False)